In [ ]:
!pip install transformers sentence-transformers faiss-cpu nltk openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 26.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving datasetN.xlsx to datasetN.xlsx


replacing any missing values in the fulltext column with an empty string to prevent error during processing (like chunking or embedding). The tolist() convert the column from a pandas series to python list


In [ ]:
df= pd.read_excel("datasetN.xlsx")
texts= df["fulltext"].fillna("").tolist()

**Chunking the text**

In [ ]:
from nltk.tokenize import sent_tokenize
def chunk_text(text, max_words=200):
  sentences = text.split('.') #sent tokenize splits the text into individual sentences using NLTK and keeps sentence boundaries intact. Manually splitting the text whenever there is a period, so each sentence ending with a ., and separating the text accordingl
  chunks= []
  chunk= "" #in this, sentences will be added on until we hit the word limit
  for sentence in sentences:
    if len(chunk.split()) + len(sentence.split()) <=max_words: #keep on adding till the sentence is under 200 words
      chunk+= " "+ sentence
    else: #if adding the sebtebce exceeds the 200-word limit, save the current chunk to the final list
        chunks.append(chunk.strip())
        chunk= sentence #start a new chunk with this sentence
  if chunk: #anything that is lefft eill be added to the last remaining chunk
    chunks.append(chunk.strip())
    return chunks

all_chunks =[]
metadata= []

for i, text in enumerate(texts):
  chunks= chunk_text(text)
  for chunk in chunks:
    all_chunks.append(chunk)
    metadata.append({
        "source":df.loc[i, "Source"],
        "topic": df.loc[i, "Topic"],
        "webtitle": df.loc[i, "webtitle"],
    })

print(f"Total chunks created: {len(all_chunks)}")

Total chunks created: 4534


**Building FAISS index**

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

model= SentenceTransformer('all-MiniLM-L6-v2')
embeddings= model.encode(all_chunks, show_progress_bar= True)

index= faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))
print(f"FAISS index built with {index.ntotal} vectors")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/142 [00:00<?, ?it/s]

FAISS index built with 4534 vectors


In [ ]:
question= "How is climate change framed in different Source?"
q_embedding= model.encode([question])

top_k= 50
_, indices = index.search(np.array(q_embedding), top_k)
retrieved_chunks= [all_chunks[i] for i in indices[0]]
retrieved_meta= [metadata[i] for i in indices[0]]

In [ ]:
from transformers import pipeline

generator = pipeline("summarization", model="facebook/bart-large-cnn")

results = []
for i, chunk in enumerate(retrieved_chunks):
    meta = retrieved_meta[i]
    summary = generator(chunk, max_length=200, min_length=60, do_sample=False)[0]['summary_text']
    bias_prompt = f"""Carefully read the following news excerpt. Identify and explain any political bias. Your analysis should highlight specific words, phrases, or framing that reflect a political leaning. State whether the excerpt leans left, right, or is neutral. Justify your reasoning clearly, using the language of the excerpt. If no clear bias is detected, say: "No significant political bias detected."

{chunk}
"""

    bias = generator(bias_prompt, max_length=200, do_sample=False)[0]['summary_text']
    results.append({
        "Index": i+1,
        "Source": meta["source"],
        "Title": meta["webtitle"],
        "Topic": meta["topic"],
        "Summary": summary,
        "Bias_Analysis": bias
    })

debug_df = pd.DataFrame(results)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Your max_length is set to 200, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Your max_length is set to 200, but your input_length is only 98. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your max_length is set to 200, but your input_length is only 54. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)
Your max_length is set to 200, but your input_length is only 132. Since this is a summarization task, where outputs sho

In [ ]:
from IPython.display import display
display(debug_df)


,Index,Source,Title,Topic,Summary,Bias_Analysis
0,1,Guardian,Gardening in a climate crisis: how to be a gre...,Climate Change,The lab’s mission is to find out how Australia...,The lab’s mission is to find out how Australia...
1,2,Guardian,Only two years left of world’s carbon budget t...,Climate Change,Solar and wind energy production is increasing...,Identify and explain any political bias. State...
2,3,Guardian,Climate crisis could hit yields of key crops e...,Climate Change,Scientists have struggled to quantify what cli...,Scientists have struggled to quantify what cli...
3,4,Guardian,The ‘silent killer’: what you need to know abo...,Climate Change,There is evidence that climate change is makin...,Climate change is making heatwaves worse by we...
4,5,Guardian,Gardening in a climate crisis: how to be a gre...,Climate Change,The Australian Drylands is “essentially a grea...,Identify and explain any political bias. State...
5,6,Guardian,Climate breakdown tripled death toll in Europe...,Climate Change,"""If you repeat the heat stress over time, the ...",Identify and explain any political bias. State...
6,7,Guardian,Climate misinformation turning crisis into cat...,Climate Change,UN secretary general António Guterres called i...,Identify and explain any political bias. State...
7,8,Guardian,Deadly weekend heat in England ‘100 times more...,Climate Change,"The extremely dry spring, combined with soarin...","The extremely dry spring, combined with soarin..."
8,9,Guardian,A useful enemy? Why Tories and Reform are call...,Climate Change,"Richard Tice, its energy spokesperson, told th...","Richard Tice, its energy spokesperson, told th..."
9,10,Guardian,How Bridget Phillipson can improve the climate...,Climate Change,Paul Atkin is a member of the National Educati...,Identify and explain any political bias. State...


In [ ]:
# debug_df.to_excel("bias_results.xlsx", index=False)
# files.download("bias_results.xlsx")


In [ ]:
import os
from getpass import getpass

# Prompt for the key securely
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")


Enter your OpenAI API key: ··········


In [ ]:
import openai
import os
import pandas as pd

def analyze_news_bias(source, list_of_topics, sentiments, summary_of_news_article):
    """
    Analyzes news summary for bias, maps sentiments to topics, and provides a bias score using OpenAI API.

    Args:
        source (str): The source of the news article.
        list_of_topics (list): A list of potential topics in the article.
        sentiments (list): A list of possible sentiments.
        summary_of_news_article (str): The summary of the news article.

    Returns:
        dict: A dictionary containing the bias analysis, mapped sentiments, and a bias score.
    """
    prompt = f"""You are an expert in classifying news and identifying bias in a news article.
    You are provided with a summary of a news article, a list of topics, and sentiments.
    Your job is to map the sentiment from the list of sentiments provided to the list of topics
    and identify bias in the provided news summary.

    Source: {source}
    List of Topics: {', '.join(list_of_topics)}
    List of Sentiments: {', '.join(sentiments)}
    News Summary: {summary_of_news_article}

    Based on the above information, provide:
    1. A brief analysis of the political bias in the news summary (leaning left, right, or neutral), with justification.
    2. A mapping of the most relevant sentiment(s) from the provided list to each relevant topic from the provided list found in the summary.
    3. A numerical bias score ranging from -1 (extreme left) to +1 (extreme right), where 0 is neutral. Provide only the numerical value for the score.
    Provide the output in a clear and concise format.
    """

    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",  # Or another suitable OpenAI model
            messages=[
                {"role": "system", "content": "You are an expert in news analysis and bias detection."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=400, # Increased max_tokens to accommodate the additional output
            temperature=0.7,
        )
        analysis = response.choices[0].message.content.strip()

        # Attempt to extract the bias score from the analysis text
        bias_score = None
        lines = analysis.split('\n')
        for line in lines:
            if "numerical bias score:" in line.lower():
                try:
                    bias_score = float(line.split(':')[-1].strip())
                except ValueError:
                    bias_score = None # Handle cases where the score is not a valid number
                break # Stop after finding the score

        return {"analysis": analysis, "bias_score": bias_score}

    except Exception as e:
        return {"error": f"An error occurred: {e}"}

# Define a list of potential sentiments
sentiments = ["empathetic", "judgemental", "neutral", "concerned", "optimistic", "critical", "supportive","fear-mongering","hatred"] # Add more sentiments as needed

# Create new columns to store the bias analysis results and the bias score
debug_df['OpenAI_Bias_Analysis'] = None
debug_df['Bias_Score'] = None # New column for the numerical bias score

# Iterate through each row of the debug_df and perform bias analysis
for index, row in debug_df.iterrows():
    source = row['Source']
    # Assuming 'Topic' column contains the relevant topics as a string or list
    # If it's a string with comma-separated topics, you might need to split it
    list_of_topics = [row['Topic']] # Assuming 'Topic' is a single topic per row
    summary_of_news_article = row['Summary']

    bias_results = analyze_news_bias(source, list_of_topics, sentiments, summary_of_news_article)

    # Store the analysis result and the bias score in the new columns
    if "analysis" in bias_results:
        debug_df.loc[index, 'OpenAI_Bias_Analysis'] = bias_results['analysis']
        debug_df.loc[index, 'Bias_Score'] = bias_results['bias_score']
    else:
        debug_df.loc[index, 'OpenAI_Bias_Analysis'] = bias_results['error']
        debug_df.loc[index, 'Bias_Score'] = None # Set score to None in case of error

display(debug_df)

,Index,Source,Title,Topic,Summary,Bias_Analysis,OpenAI_Bias_Analysis,Bias_Score
0,1,Guardian,Gardening in a climate crisis: how to be a gre...,Climate Change,The lab’s mission is to find out how Australia...,The lab’s mission is to find out how Australia...,1. Political Bias Analysis:\n The news summa...,None
1,2,Guardian,Only two years left of world’s carbon budget t...,Climate Change,Solar and wind energy production is increasing...,Identify and explain any political bias. State...,1. Analysis of Political Bias:\n The news su...,-0.7
2,3,Guardian,Climate crisis could hit yields of key crops e...,Climate Change,Scientists have struggled to quantify what cli...,Scientists have struggled to quantify what cli...,1. The news summary from the Guardian appears ...,None
3,4,Guardian,The ‘silent killer’: what you need to know abo...,Climate Change,There is evidence that climate change is makin...,Climate change is making heatwaves worse by we...,1. Political Bias Analysis:\n The news summa...,-0.8
4,5,Guardian,Gardening in a climate crisis: how to be a gre...,Climate Change,The Australian Drylands is “essentially a grea...,Identify and explain any political bias. State...,1. The news summary from The Guardian appears ...,0.0
5,6,Guardian,Climate breakdown tripled death toll in Europe...,Climate Change,"""If you repeat the heat stress over time, the ...",Identify and explain any political bias. State...,1. Analysis of Political Bias:\n The news su...,None
6,7,Guardian,Climate misinformation turning crisis into cat...,Climate Change,UN secretary general António Guterres called i...,Identify and explain any political bias. State...,1. Political Bias Analysis:\nThe news summary ...,0.6
7,8,Guardian,Deadly weekend heat in England ‘100 times more...,Climate Change,"The extremely dry spring, combined with soarin...","The extremely dry spring, combined with soarin...",1. The news summary about the high risk of wil...,None
8,9,Guardian,A useful enemy? Why Tories and Reform are call...,Climate Change,"Richard Tice, its energy spokesperson, told th...","Richard Tice, its energy spokesperson, told th...",1. The news summary exhibits a potential bias ...,-0.6
9,10,Guardian,How Bridget Phillipson can improve the climate...,Climate Change,Paul Atkin is a member of the National Educati...,Identify and explain any political bias. State...,1. Political Bias Analysis:\nThe news summary ...,0.0


In [ ]:
# debug_df.to_excel("biasopenai_results.xlsx", index=False)
# files.download("biasopenai_results.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now, let's apply the same process to the "Immigration" and "Israel-Palestine" topics.

In [ ]:
# Filter the original DataFrame for the new topics
immigration_df = df[df['Topic'] == 'Immigration'].copy()
israel_palestine_df = df[df['Topic'] == 'Israel-Palestine'].copy()

# Create chunks and metadata for Immigration
immigration_texts = immigration_df["fulltext"].fillna("").tolist()
all_immigration_chunks = []
immigration_metadata = []

for i, text in enumerate(immigration_texts):
    chunks = chunk_text(text)
    for chunk in chunks:
        all_immigration_chunks.append(chunk)
        immigration_metadata.append({
            "source": immigration_df.loc[immigration_df.index[i], "Source"],
            "topic": immigration_df.loc[immigration_df.index[i], "Topic"],
            "webtitle": immigration_df.loc[immigration_df.index[i], "webtitle"],
        })

print(f"Total chunks created for Immigration: {len(all_immigration_chunks)}")

# Create chunks and metadata for Israel-Palestine
israel_palestine_texts = israel_palestine_df["fulltext"].fillna("").tolist()
all_israel_palestine_chunks = []
israel_palestine_metadata = []

for i, text in enumerate(israel_palestine_texts):
    chunks = chunk_text(text)
    for chunk in chunks:
        all_israel_palestine_chunks.append(chunk)
        israel_palestine_metadata.append({
            "source": israel_palestine_df.loc[israel_palestine_df.index[i], "Source"],
            "topic": israel_palestine_df.loc[israel_palestine_df.index[i], "Topic"],
            "webtitle": israel_palestine_df.loc[israel_palestine_df.index[i], "webtitle"],
        })

print(f"Total chunks created for Israel-Palestine: {len(all_israel_palestine_chunks)}")

Total chunks created for Immigration: 1149
Total chunks created for Israel-Palestine: 1887


Now, let's build FAISS indices and retrieve relevant chunks for each topic.

In [ ]:
# Build FAISS index for Immigration
immigration_embeddings = model.encode(all_immigration_chunks, show_progress_bar=True)
immigration_index = faiss.IndexFlatL2(immigration_embeddings.shape[1])
immigration_index.add(np.array(immigration_embeddings))
print(f"FAISS index built for Immigration with {immigration_index.ntotal} vectors")

# Build FAISS index for Israel-Palestine
israel_palestine_embeddings = model.encode(all_israel_palestine_chunks, show_progress_bar=True)
israel_palestine_index = faiss.IndexFlatL2(israel_palestine_embeddings.shape[1])
israel_palestine_index.add(np.array(israel_palestine_embeddings))
print(f"FAISS index built for Israel-Palestine with {israel_palestine_index.ntotal} vectors")

Batches:   0%|          | 0/36 [00:00<?, ?it/s]

FAISS index built for Immigration with 1149 vectors


Batches:   0%|          | 0/59 [00:00<?, ?it/s]

FAISS index built for Israel-Palestine with 1887 vectors


Now, let's perform the bias analysis for each topic using the retrieved chunks and the OpenAI API.

In [ ]:
# Define the question for retrieval for each topic
immigration_question = "How is immigration framed in different sources?"
israel_palestine_question = "How is the Israel-Palestine conflict framed in different sources?"

# Retrieve chunks for Immigration
q_embedding_immigration = model.encode([immigration_question])
_, indices_immigration = immigration_index.search(np.array(q_embedding_immigration), top_k)
retrieved_immigration_chunks = [all_immigration_chunks[i] for i in indices_immigration[0]]
retrieved_immigration_meta = [immigration_metadata[i] for i in indices_immigration[0]]

# Retrieve chunks for Israel-Palestine
q_embedding_israel_palestine = model.encode([israel_palestine_question])
_, indices_israel_palestine = israel_palestine_index.search(np.array(q_embedding_israel_palestine), top_k)
retrieved_israel_palestine_chunks = [all_israel_palestine_chunks[i] for i in indices_israel_palestine[0]]
retrieved_israel_palestine_meta = [israel_palestine_metadata[i] for i in indices_israel_palestine[0]]

# Perform bias analysis for Immigration
immigration_results = []
for i, chunk in enumerate(retrieved_immigration_chunks):
    meta = retrieved_immigration_meta[i]
    summary = generator(chunk, max_length=200, min_length=60, do_sample=False)[0]['summary_text']
    bias_prompt = f"""Carefully read the following news excerpt. Identify and explain any political bias. Your analysis should highlight specific words, phrases, or framing that reflect a political leaning. State whether the excerpt leans left, right, or is neutral. Justify your reasoning clearly, using the language of the excerpt. If no clear bias is detected, say: "No significant political bias detected."

{chunk}
"""
    bias_results = analyze_news_bias(meta["source"], [meta["topic"]], sentiments, summary)
    immigration_results.append({
        "Index": i+1,
        "Source": meta["source"],
        "Title": meta["webtitle"],
        "Topic": meta["topic"],
        "Summary": summary,
        "OpenAI_Bias_Analysis": bias_results['analysis'] if 'analysis' in bias_results else bias_results['error'],
        "Bias_Score": bias_results['bias_score'] if 'bias_score' in bias_results else None
    })

immigration_df_results = pd.DataFrame(immigration_results)
print("Bias analysis results for Immigration:")
display(immigration_df_results)

# Perform bias analysis for Israel-Palestine
israel_palestine_results = []
for i, chunk in enumerate(retrieved_israel_palestine_chunks):
    meta = retrieved_israel_palestine_meta[i]
    summary = generator(chunk, max_length=200, min_length=60, do_sample=False)[0]['summary_text']
    bias_prompt = f"""Carefully read the following news excerpt. Identify and explain any political bias. Your analysis should highlight specific words, phrases, or framing that reflect a political leaning. State whether the excerpt leans left, right, or is neutral. Justify your reasoning clearly, using the language of the excerpt. If no clear bias is detected, say: "No significant political bias detected."

{chunk}
"""
    bias_results = analyze_news_bias(meta["source"], [meta["topic"]], sentiments, summary)
    israel_palestine_results.append({
        "Index": i+1,
        "Source": meta["source"],
        "Title": meta["webtitle"],
        "Topic": meta["topic"],
        "Summary": summary,
        "OpenAI_Bias_Analysis": bias_results['analysis'] if 'analysis' in bias_results else bias_results['error'],
        "Bias_Score": bias_results['bias_score'] if 'bias_score' in bias_results else None
    })

israel_palestine_df_results = pd.DataFrame(israel_palestine_results)
print("\nBias analysis results for Israel-Palestine:")
display(israel_palestine_df_results)

Your max_length is set to 200, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 200, but your input_length is only 112. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)
Your max_length is set to 200, but your input_length is only 190. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=95)
Your max_length is set to 200, but your input_length is only 123. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)
Yo

Bias analysis results for Immigration:


,Index,Source,Title,Topic,Summary,OpenAI_Bias_Analysis,Bias_Score
0,1,Guardian,Northern Ireland politicians condemn migrants ...,Immigration,"""This year the focus is on the scandal of mass...",1. Analysis of Political Bias:\nThe news summa...,-0.6
1,2,cnn,Trump’s visit to a migrant camp called ‘Alliga...,Immigration,The backlog of cases is partly the product of ...,1. Analysis of Political Bias:\n The news su...,-0.5
2,3,breitbart-news,Pro-American Migration Reform Groups Spar over...,Immigration,"NumbersUSA is for lower immigration, he said. ...",1. Analysis of Political Bias:\nThe news summa...,0.7
3,4,cnn,California’s immigration raids in Los Angeles:...,Immigration,A federal judge found the Department of Homela...,1. Analysis of Political Bias:\n Based on th...,-0.6
4,5,cnn,Less than 10% of immigrants taken into ICE cus...,Immigration,Internal government documents obtained by CNN ...,1. Political Bias Analysis:\nThe news summary ...,-0.5
5,6,breitbart-news,Marc Andreesen: 'No Chance' for Smart American...,Immigration,Andreessen’s viewpoint about the impact of mig...,1. Analysis of Political Bias:\n The news su...,-0.7
6,7,breitbart-news,White House Shoots Down Salazar's Amnesty and ...,Immigration,Salazar’s bill is backed by “the unholy allian...,1. The news summary from Breitbart News exhibi...,NaN
7,8,cnn,Trump is creating a new universe of people to ...,Immigration,People who were working legally are now undocu...,1. Bias Analysis:\nThe news summary leans towa...,NaN
8,9,breitbart-news,Trump Cuts Biden's Parole Scheme by 100% as Ze...,Immigration,President Donald Trump’s Department of Homelan...,1. Political Bias Analysis:\nThe news summary ...,0.8
9,10,cnn,Trump is creating a new universe of people to ...,Immigration,The Trump administration has been cracking dow...,1. Political Bias Analysis:\nThe news summary ...,NaN


Your max_length is set to 200, but your input_length is only 52. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)
Your max_length is set to 200, but your input_length is only 137. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=68)
Your max_length is set to 200, but your input_length is only 187. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=93)
Your max_length is set to 200, but your input_length is only 174. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=87)



Bias analysis results for Israel-Palestine:


,Index,Source,Title,Topic,Summary,OpenAI_Bias_Analysis,Bias_Score
0,1,breitbart-news,Mike Huckabee Visits Palestinian Christians; O...,Israel-Palestine,The problem appears to have been caused by a s...,1. Political Bias Analysis:\nThe news summary ...,NaN
1,2,breitbart-news,Wilders to Macron: Create New Palestinian Stat...,Israel-Palestine,Wilders has consistently positioned the Jewish...,1. Analysis of Political Bias:\nThe news summa...,0.7
2,3,Guardian,I call on the UK to join the world majority – ...,Israel-Palestine,The genocide in Gaza has exposed Israel’s inte...,1. **Political Bias Analysis:**\n The news s...,NaN
3,4,breitbart-news,Mike Huckabee Visits Palestinian Christians; O...,Israel-Palestine,Huckabee knows that Israel cares deeply about ...,1. Analysis of Political Bias:\nThe news summa...,0.7
4,5,breitbart-news,Israel Tries to Stop Actual Genocide in Syria ...,Israel-Palestine,Israel has provided protection to the Gaza Hum...,1. Political Bias Analysis:\nThe news summary ...,0.7
5,6,Guardian,‘Heads will roll’: BBC reckons with bias accus...,Israel-Palestine,The BBC is set to publish an investigation int...,1. Bias Analysis:\n The news summary from Th...,0.2
6,7,Guardian,‘Heads will roll’: BBC reckons with bias accus...,Israel-Palestine,The BBC has not been able to deliver satisfact...,1. Bias Analysis:\nThe news summary appears to...,0.0
7,8,Guardian,‘Post-apocalyptic’: medical staff struggle as ...,Israel-Palestine,The Israel Defense Forces (IDF) holds much of ...,1. Analysis of Political Bias:\nThe news summa...,NaN
8,9,Guardian,The Gaza discourse has been Vylanised – but th...,Israel-Palestine,At least 400 Palestinians have been killed rec...,1. Political Bias Analysis:\nThe news summary ...,-0.7
9,10,Guardian,The BBC has alienated everyone with its Gaza c...,Israel-Palestine,Muslim Council of Britain’s Centre for Media M...,1. Analysis of Political Bias:\nThe news summa...,-0.7


In [ ]:
immigration_df_results.to_excel("biasopenai_results_immigration.xlsx", index=False)
files.download("biasopenai_results_immigration.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
israel_palestine_df_results.to_excel("biasopenai_results_israelpalestine.xlsx", index=False)
files.download("biasopenai_results_israelpalestine.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>